In [1]:
import requests
year = 2022
day = 17

url = f"https://adventofcode.com/{year}/day/{day}/input"

with open('cookie.txt', 'r') as file:
    session = file.read()

headers = {'cookie': f"session={session}"}

with open(f'day_{day}_input.txt', 'wb') as file:
    file.write(requests.get(url, headers=headers).content)
    
import os
if os.path.exists(f"day_{day}_test.txt"):
    test = [x.strip() for x in open(f'day_{day}_test.txt', 'r')]
    for line in test[:10]:
        print(line.strip())
print("---")
if os.path.exists(f"day_{day}_test_2.txt"):
    test_2 = [x.strip() for x in open(f'day_{day}_test_2.txt', 'r')]
    for line in test[:10]:
        print(line.strip())
print("---")
lines = [x.strip() for x in open(f'day_{day}_input.txt', 'r')]
for line in lines[:10]:
    print(line.strip())

>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>
---
---
>>><<<<>>><<<<><<<<>><><>>>><<><<>>>><<>>>><<<>>>><<<>>><<<<>><<><>>><><<>><<>><<><>>><>>>><<<><<<>>><<<<>><<<<>>>><<>>>><<>><<<<>><><<<><<<<>>><<<<>>>><<<>>><<<>>><>><<><<<<>>>><<<><>>>><<<<>>><<><<<<>><<<<>>>><><<<<>>>><<<<>><<<<><<<>>>><<<><<<>><<>>>><<<>><<<<>>>><<<>>>><>><<<<>>><<><>>><>>>><<>>>><<>>>><>>>><>><<>><<><>>>><<>>><<>>>><<<<>>>><>><>>>><>>>><<<>><<>>>><<<<><<<<><<<>><<<>>><<<<><<<<>>><<<>><<<>>><<<<>>><<<>><<<>>>><>>>><<<>>>><<<<>><<>>><<<>>>><<>>>><<<<>>><<<<><>>>><>>>><<>>>><>>><<<<>>><<<>>><<<<><>><<<<>><<>>><<>>><>>>><<<<>>><<>>>><>>>><>><<<<>>>><<<<>>><<<>>>><<>><<<<>><<>>><>>>><<<>>>><>>><<<>>>><>><<<<>>><>><<<><<<<>>>><>>>><>><<>><<><<><><><<<<>>>><><<>>><>><<><<<>>>><<<>><<<<>>><<<<><<<>>>><><<>>><>>><<<<>><<<>>><<<<><<<<><<<>><><><<<>>><>><><<<<>><<<><><<>>><<<>>><<<<>><<<<>><<<>><>>>><><<<<>>>><<<<>><<<<><<>><<<>>>><><<>>>><<><<><>><<<<>><>><<<><<<>>><<>><<<<>>><<<<>><<<<><<<><><>>>><<<<>>><<<>>><<<>>><<<>>>><>

In [2]:
import numpy as np
from pprint import pprint


run_test = False

if run_test:
    case = test
else:
    case = lines
    
# for line in case:
#     print(line)
    
rock_spawns = (
    ((0,0),(1,0),(2,0),(3,0)),
    ((0,1),(1,0),(1,1),(2,1),(1,2)),
    ((0,2),(1,2),(2,2),(2,1),(2,0)),
    ((0,0),(0,1),(0,2),(0,3)),
    ((0,0),(0,1),(1,0),(1,1))
)

M_ROCK = 2
SPACE = 0
STOPPED = 1

ROOM_WIDTH = 7
empty_row = np.zeros(shape=(1,ROOM_WIDTH))
room = empty_row.copy()
#room = []

def spawn_rock(i=0):
    global room
    
    height_required = height_of_tower(room) + 3 + height_of_rock(i)
    
    while room.shape[0] < height_required:
        room = np.vstack((empty_row,room))
    while room.shape[0] > height_required:
        room = room[1:,:]
                
    for point in rock_spawns[i]:
        offset_x = 2
        room[point[1]][point[0]+offset_x] = M_ROCK
    

def height_of_tower(room):
    rocks = np.where(room == STOPPED)
    if len(rocks[0]) == 0:
        h = 0
        #print(f"tower: {h}")
        return h
    else:
        h = max(rocks[0]) - min(rocks[0]) + 1
        #print(f"tower: {h}")
        return h
    
def height_of_rock(i):
    h = max([x[1] for x in rock_spawns[i]]) + 1
    #print(f"rock: {h}")
    return h

def move_rock(direction):
    global room
    n_room = room.copy()
    
    rocks = list(zip(*np.where(room == M_ROCK)))

    if direction == '>':
        offset = 1
        rocks = reversed(rocks)
    else:
        offset = -1
        
                
    for i, j in rocks:
        if not (0 <= j+offset < ROOM_WIDTH):
            return False
        if n_room[i,j+offset] == STOPPED:
            return False
        
        n_room[i,j] = SPACE
        n_room[i,j+offset] = M_ROCK
    
    room = n_room.copy()
    
    return True

def fall_rock():
    global room
    n_room = room.copy()
    
    rocks = reversed(list(zip(*np.where(room == M_ROCK))))
                
    for i, j in rocks:
        if not (0 <= i+1 < room.shape[0]):
            return False
        if n_room[i+1,j] == STOPPED:
            return False
        
        n_room[i,j] = SPACE
        n_room[i+1,j] = M_ROCK
    
    room = n_room.copy()
    
    return True


def print_room(room):
    
    map_ = {
        M_ROCK:'@',
        SPACE: '.',
        STOPPED: '#'
    }
    
    for i, row in enumerate(room):
        # f'
        print(f'{i:04d}|' + ''.join([map_[x] for x in row]) + '|' + str(sum([(2)**j for j, d in enumerate(reversed(row)) if d == 1])))
    print('    +-------+')
    

def deepest_reachable():
    global room
    indices_found = set()
    for i, row in enumerate(room):
        indices_here = np.where(row == STOPPED)[0]
        indices_found.update(indices_here)
        if len(indices_here) == ROOM_WIDTH:
            return i
    return room.shape[0]

from tqdm import tqdm

r = 0
drn = 0
max_tower = 0

states = {}

height_after_rock = {}

depths = set()

ROCKS = 1000000000000

jumped = False


i = 0
while i < (ROCKS):
    spawn_rock(r)
    
    
    if not jumped:
        room_tuple = tuple(sum([(2)**j for j, d in enumerate(reversed(row)) if d == 1]) for row in room[:30])
        #print(room_tuple)
        state = (drn, r, room_tuple)
        if state in states:
            rocks_since_repeated = i - states[state][0]
            height_since_repeated = height_of_tower(room) - states[state][1]

            repetitions = (ROCKS - i) // rocks_since_repeated

            max_tower = repetitions * height_since_repeated

            i = i + repetitions * rocks_since_repeated
            print(i)
            jumped = True

        else:
            states[state] = [i, height_of_tower(room)]
            
    #if not jumped:
    i += 1

    r = (r + 1)%len(rock_spawns)

    
#     if i >= 380:
#         print_room(room[:40,:])
    while True:
        moved = move_rock(case[0][drn])
        drn = (drn + 1)%len(case[0])
        #print("move")
        #print_room()

        #print("fall")
        fell = fall_rock()
        #print_room()

        if not fell:
            #print("stop")
            room[room == M_ROCK] = STOPPED
            break
    height_after_rock[i] = height_of_tower(room)
    #print_room()
#     deepest = deepest_reachable()
#     if deepest < room.shape[0]:
#         max_tower += room.shape[0]-deepest
#         room = room[:deepest,:]  

print(i)
print(max_tower + height_of_tower(room))


999999998513
1000000000000
1555113636385
